# Setting up Selenium for Uber 

In [ ]:
from selenium import webdriver
browser = webdriver.Chrome("path-to-chromedriver")
browser.get('https://drivers.uber.com/p3/payments/trips/{your-trip-id}')

In [ ]:
username = browser.find_element_by_id('useridInput')
username.send_keys('enter-your-email-here')
Next = browser.find_element_by_class_name('push-small--right')
Next.click()

In [ ]:
password = browser.find_element_by_id('password')
password.send_keys('enter-your-password-here')
Next = browser.find_element_by_class_name('push-small--right')
Next.click()

In [ ]:
verification = browser.find_element_by_id('verificationCode')
verification.send_keys('enter-your-verification-code-here')
Next = browser.find_element_by_class_name('push-small--right')
Next.click()

# Scrapping Uber Trip Data

In [ ]:
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
import os

path = 'path-to-statements-folder-here'
files = os.listdir(path)

pay_df = []
date_df = []
duration_df = []
distance_df = []
fare_df = []
surge_df = []
surge_fee_df = []
UberFee_df = []
tripid_df = []
pickup_df = []
dropoff_df = []

for x in range(0,len(files)):
    
    # Excessing files within relevant folder
    f_path = os.path.join(path, files[x])    
    
    # Uploding the uber statement file and segragaing the tripID column
    statement = pd.read_csv(f_path)
    tripID  = statement['Trip ID']

    # Scrapping all the relevant trip data
    for tid in tripID:
        # creating weblink to fetch information and pushing it onto chrome
        weblink = os.path.join('https://drivers.uber.com/p3/payments/trips/',tid)
        
        #delaying execution
        time.sleep(1)
        browser.get(weblink)

        # scrapping the page using beautiful soap
        page_source = browser.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        pay = soup.find('h1', class_='bi br').get_text()
        date = soup.find('div', class_='cs ct cu cv cw cx cy').get_text()
        duration = soup.find_all('h4', class_='bi')[0].get_text()
        distance = soup.find_all('h4', class_='bi')[1].get_text()
        fare = soup.find_all('span', class_='c0 c1 d9 bj')[0].get_text()
        pickup = soup.find_all('div', class_="dr")[0].get_text()
        loc = soup.find_all('div', class_="dq")[0].get_text()
        dropoff = loc.replace(pickup,"")
        tripid = tid
        try:
            UberFee = soup.find_all('span', class_='c0 c1 d9')[1].get_text()
            surge = soup.find_all('span', class_='c0 c1 d9')[0].get_text()
            surge_fee = "Yes"
        except:
            UberFee = soup.find_all('span', class_='c0 c1 d9')[0].get_text()
            surge = "NA"
            surge_fee = "No"

        # appending results to arrays
        pay_df.append(pay)
        date_df.append(date)
        duration_df.append(duration)
        distance_df.append(distance)
        fare_df.append(fare)
        surge_df.append(surge)
        UberFee_df.append(UberFee)
        pickup_df.append(pickup)
        dropoff_df.append(dropoff)
        tripid_df.append(tripid)
        surge_fee_df.append(surge_fee)

    # Enquiring Status
    print("The file no " + str(x+1) + " named as " + files[x] + " is completed out of " + str(len(files)))
browser.quit()

In [ ]:
uber_data = { 'TripID' : tripid_df,
              'Date and Time' : date_df,
              'Surge Status' : surge_fee_df,
              'Fare' : fare_df,
              'Surge' : surge_df,
              'UberFee' : UberFee_df,
              'Pay' : pay_df,
              'Duration' : duration_df,
              'Distance' : distance_df,
              'Pickup Time' : pickup_df,
              'Dropoff Time' : dropoff_df}

uber_data_df = pd.DataFrame(uber_data)
uber_data_df.to_csv('path-to-output-file')

In [ ]:
uber_data_df